In [1]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
from pathlib import Path
import shutil

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))


import sys
sys.path.append(  "/notebook/nas-trainings/arne/DGFISMA/definition_extraction/notebooks/" )

In [2]:
#Good use case for definition extraction:

#https://eur-lex.europa.eu/eli/reg_impl/2018/1627/oj


### DGFISMA DATA:

In [3]:
#DGFISMA_DATA:
ANNEX_regexp= r'^(ANNEX)\ *([0-9.]|ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN|EIGHT|NINE|TEN|ELEVEN|TWELVE|THIRTEEN|FOURTEEN|FIFTEEN|SIXTEEN|SEVENTEEN|EIGHTEEN|NINETEEN|TWENTY|I|II|III|IV|V|VI|VII|VIII|IX|X|X(I|II|III|IV|V|VI|VII|VIII|IX|X))* *$'

In [4]:
import glob
import json
import base64
from cleaning import clean_html, delete_annexes

DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_11_06_DGFISMA/"
files=glob.glob( os.path.join( DATA_DIR , "*" )  )

all_articles=[]

for i, file in enumerate(files):

    if os.path.getsize( file ) > 20000000:
        print(  f"{file} is larger than 20Mb" )
        continue
    
    jsonl=open( file ).read().rstrip("\n").split( "\n" )
    json_file=json.loads( jsonl[0])
 
    print( i, json_file['ELI'] )
    
    decoded_content=base64.b64decode( json_file['content']  ).decode( 'utf-8' )

    articles=clean_html( decoded_content )
    articles=delete_annexes(articles)

    articles_text="\n".join( articles )
    articles_text.replace( "\n", " "  )
    
    all_articles.append(  articles_text.replace( "\n", " "  ) )
    
    '''
    articles=clean_html( decoded_content )

    #delete the annexes:
    articles_no_annexes=[]
    for article in articles:
        if bool(re.match( ANNEX_regexp,  article.strip("\n").split( "\n" )[0]  , re.IGNORECASE )):
            #continue
            break  #not adding annexes, or anything that comes after annex to the training set
        else:
            articles_no_annexes.append( article )
    
    if articles_no_annexes:
        delimiter=100*"■"
        article_no_annexes=delimiter.join(  articles_no_annexes  )
        all_articles.append( article_no_annexes )
    '''


0 ['http://data.europa.eu/eli/dec_impl/2019/1874/oj']
1 ['http://data.europa.eu/eli/dec_impl/2019/1277/oj']
2 ['http://data.europa.eu/eli/dec_impl/2019/1280/oj']
3 ['http://data.europa.eu/eli/dec_impl/2019/1275/oj']
4 ['http://data.europa.eu/eli/dec_impl/2019/1274/oj']
5 ['http://data.europa.eu/eli/dec_impl/2019/1282/oj']
6 ['http://data.europa.eu/eli/dec_impl/2019/1279/oj']
7 ['http://data.europa.eu/eli/dec_impl/2019/1276/oj']
8 ['http://data.europa.eu/eli/dec_impl/2019/1281/oj']
9 ['http://data.europa.eu/eli/dec_impl/2019/1278/oj']
10 ['http://data.europa.eu/eli/dec_impl/2019/1283/oj']
11 ['http://data.europa.eu/eli/dec_impl/2019/684/oj']
12 ['http://data.europa.eu/eli/dec_impl/2019/544/oj']
13 ['http://data.europa.eu/eli/dec_impl/2019/541/oj']
14 ['http://data.europa.eu/eli/dec_impl/2019/536/oj']
15 ['http://data.europa.eu/eli/dec_impl/2018/2047/oj']
16 ['http://data.europa.eu/eli/dec_impl/2018/2030/oj']
17 ['http://data.europa.eu/eli/dec_impl/2018/2031/oj']
18 ['http://data.europa.

152 ['http://data.europa.eu/eli/reg_impl/2018/708/oj']
153 ['http://data.europa.eu/eli/reg_impl/2018/634/oj']
154 ['http://data.europa.eu/eli/reg_impl/2018/633/oj']
155 ['http://data.europa.eu/eli/reg_del/2018/541/oj']
156 ['http://data.europa.eu/eli/reg_del/2018/480/oj']
157 ['http://data.europa.eu/eli/reg/2018/498/oj']
158 ['http://data.europa.eu/eli/reg_del/2018/389/oj']
159 ['http://data.europa.eu/eli/reg_del/2018/344/oj']
160 ['http://data.europa.eu/eli/reg_impl/2018/292/oj']
161 ['http://data.europa.eu/eli/reg_impl/2018/165/oj']
162 ['http://data.europa.eu/eli/reg/2018/182/oj']
163 ['http://data.europa.eu/eli/reg_del/2018/67/oj']
164 ['http://data.europa.eu/eli/reg_del/2018/66/oj']
165 ['http://data.europa.eu/eli/reg_del/2017/2358/oj']
166 ['http://data.europa.eu/eli/reg_del/2017/2359/oj']
167 ['http://data.europa.eu/eli/reg_del/2017/2361/oj']
168 ['http://data.europa.eu/eli/reg_del/2017/2295/oj']
169 ['http://data.europa.eu/eli/reg_impl/2017/2241/oj']
170 ['http://data.europa.eu

In [5]:
data=[]
labels=len(all_articles)*[1]
data=all_articles

## Non DGFISMA Data

In [6]:
import glob
import json
DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_30_04"
files=glob.glob( os.path.join( DATA_DIR, "EURLEX" , "*jsonl" )  )

all_jsons=[]

xhtmls=[]
htmls=[]
pdfs=[]

for file in files:
    jsonl=open( file ).read().rstrip("\n").split( "\n" )
    json_file = [json.loads( json_line ) for json_line in jsonl]
    if len( json_file ) >1:
        if json_file[1][ "content_type" ]=="xhtml":
            xhtmls.append( json_file  )
        
        if json_file[1][ "content_type" ]=="html":
            htmls.append( json_file )
        
        if json_file[1][ "content_type" ]=="pdf":
            pdfs.append( json_file )
            

In [7]:
import glob
import json
import base64
from cleaning import clean_html

non_dgfisma=[1044, 482, 2029, 1841, 1934, 2668, 1554, 859, 384, 1998, 1596, 1772, 2488, 1824, 1090, 937, 2421, 418, 1300, 125, 91, 104, 37, 1561, 887 , 118, 2161, 908, 1793, 2264,954 ,1415, 945, 896,1882, 1186,88 \
          ,2279, 409, 761, 2577, 1214, 495, 1362, 2051, 2079, 777, 1242, 1163, 2406, 2069, 2412, 141, 1967, 994, 1655, 2722, 1503, 2247, 1534, 354, 2718, 2082, 670, 2133, 1610, 1517, 2005, 121, 178, 1263, \
           2518, 2429, 2368,1612, 697, 2057, 929, 50, 817, 2210, 2245, 950, 1656, 2104, 1408, 2366, 1447, 1102, 2700, 2244, 2494, 23, 1571, 2099, 529, 2124, 2299, 841, 1745, 229, 1970, 1493, 2334, 2270, 2067, \
         1693, 1986, 1461, 1697, 1417, 6, 2205, 2553, 2508, 1356, 1876, 2457, 114, 940, 2602, 725, 2255, 2393, 740, 375, 1045, 132, 288, 340, 68, 1855, 59, 1151, 1022, 1100, 448, 1189, 284, 653, 1045, 2160, \
         2689, 1117, 2655, 1206, 1862, 1318, 2033, 1940, 467, 96, 1277, 1583, 1406, 1724, 1058, 445, 1038, 2089, 856, 2480, 1768, 85, 923, 1627, 656, 2073, 1747, 2231, 903, 2115, 1846, 914, 2145, 2656, 125, \
         1617, 2358, 1315, 2584, 1746, 240, 1223, 514, 868, 289, 313, 1271, 1220, 648, 1704, 2313, 1033, 534, 34, 2296, 155, 2419, 891, 2335, 1887, 702, 2551, 2084, 153, 1548, 820, 1421, 842, 1773, 795, 2016, \
         427, 2727, 1212, 2064, 2047, 70, 1332, 2507, 1647, 1152 , 74 , 642, 822, 1342, 553, 1388, 872, 394, 1553, 2243, 1408, 1984, 2181, 916, 961, 267, 165, 346, 695, 682, 2204, 1097, 1360,1507, 1387, 1393, \
         936, 554, 2375]

all_articles=[]

for i, json_file in enumerate( xhtmls ):

    if i not in non_dgfisma:
        continue
    
    #if os.path.getsize( file ) > 20000000:
    #    print(  f"{file} is larger than 20Mb" )
    #    continue
    
    #jsonl=open( file ).read().rstrip("\n").split( "\n" )
    #json_file=json.loads( jsonl[0])
 
    print( i, json_file[0]['url'] )
    
    decoded_content=base64.b64decode( json_file[1]['content']  ).decode( 'utf-8' )

    
    articles=clean_html( decoded_content )
    articles=delete_annexes(articles)

    articles_text="\n".join( articles )
    articles_text.replace( "\n", " "  )
    
    all_articles.append(  articles_text.replace( "\n", " "  ) )
    
    '''
    
    articles=clean_html( decoded_content )

    #delete the annexes:
    articles_no_annexes=[]
    for article in articles:
        if bool(re.match( ANNEX_regexp,  article.strip("\n").split( "\n" )[0]  , re.IGNORECASE )):
            #continue
            break  #not adding annexes, or anything that comes after annex to the training set
        else:
            articles_no_annexes.append( article )
    
    if articles_no_annexes:
        delimiter=100*"■"
        article_no_annexes=delimiter.join(  articles_no_annexes  )
        all_articles.append( article_no_annexes )
        
    '''

6 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32012D0378']
23 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32016R1356']
34 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32016L1106']
37 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32005L0015']
50 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32019L1813']
59 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32005L0021']
68 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32017D1995']
70 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32008D0154']
74 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:22010D0239']
85 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32007L0054']
88 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32012L0049']
91 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32014B0545']
96 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32017L1975']
104 ['https:/

1277 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32010D0228']
1300 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32007R0124']
1315 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32017D0695']
1318 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32010L0077']
1332 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32013L0019']
1342 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32010L0048']
1356 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32016L0798']
1360 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32011L0071']
1362 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:22017D2029']
1387 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32014L0081']
1388 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32009R0315']
1393 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32007R0821']
1406 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX

2393 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32006L0084']
2406 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32008L0032']
2412 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32020L0364']
2419 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32012L0037']
2421 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32011R1136']
2429 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32007R1539']
2457 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32013R0734']
2480 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32008L0017']
2488 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32020D0010']
2494 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32004L0104']
2507 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32010L0023']
2508 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32007L0034']
2518 ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX

In [9]:
labels=labels+len( all_articles )*[0]
data=data+all_articles

In [11]:
import random
from base64 import b64encode
from base64 import b64decode

#import html2text
random.seed( 10 )


labels_text=[]
for label, text in zip( labels, data  ):
    labels_text.append(  ( label, text  ) )
    

random.shuffle( labels_text )

print( len(labels_text) )

train_set=labels_text[:400]
test_set=labels_text[ 400: ]


with open( os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06", "train_data.tsv"  ) , 'w'  ) as f:
    for item in train_set:
        if item[0]==1:
            label_name='accepted'
        else:
            label_name='declined'

        encoded_doc=b64encode(  item[1].encode() )
        f.write( f"{encoded_doc.decode()  }\t{ label_name }\t{item[0]}\n" )
        
        
with open( os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06", "test_data.tsv"  ) , 'w'  ) as f:
    for item in test_set:
        if item[0]==1:
            label_name='accepted'
        else:
            label_name='declined'

        encoded_doc=b64encode(  item[1].encode() )
        f.write( f"{encoded_doc.decode()  }\t{ label_name }\t{item[0]}\n" )



493


In [13]:
#also write in csv format:
import os
import pandas as pd
from base64 import b64encode
from base64 import b64decode
import base64

filename=os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06", "test_data.tsv"  )

data=pd.read_csv(  filename  , sep='\t' , header=None ) 

train_data=data[0].tolist()
train_label_names=data[1].tolist()
train_labels=data[2].tolist()


train_data=[ b64decode( doc ).decode() for doc in train_data  ]

with open( os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06" , "test_data.csv"  ) , 'w'  ) as f:
    for doc, label_name, label  in zip( train_data , train_label_names, train_labels  ):
        f.write( f"\"{ doc  }\"⚫{ label_name }⚫{  label }\n" )

In [37]:

filename=os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06", "test_data_milieu_iteration1.tsv"  )

data=pd.read_csv(  filename  , sep='\t' , header=None ) 

train_data=data[0].tolist()
train_label_names=data[1].tolist()
train_labels=data[2].tolist()

delimiter="■"*100

train_data=[ b64decode( doc ).decode().replace( "\n", ". " ).replace( delimiter, " " ) for doc in train_data  ]

train_data[1].split(  )[202:202+512]

['to',
 'the',
 'recapitalisation',
 'and',
 'restructuring',
 'of',
 'Piraeus',
 'Bank',
 'SA.',
 '(notified',
 'under',
 'document',
 'C(2014)',
 '5217).',
 '(Only',
 'the',
 'English',
 'text',
 'is',
 'authentic).',
 '(Text',
 'with',
 'EEA',
 'relevance).',
 'THE',
 'EUROPEAN',
 'COMMISSION,.',
 'Having',
 'regard',
 'to',
 'the',
 'Treaty',
 'on',
 'the',
 'Functioning',
 'of',
 'the',
 'European',
 'Union,',
 'and',
 'in',
 'particular',
 'the',
 'first',
 'subparagraph',
 'of',
 'Article',
 '108(2)',
 'thereof,.',
 'Having',
 'regard',
 'to',
 'the',
 'Agreement',
 'on',
 'the',
 'European',
 'Economic',
 'Area,',
 'and',
 'in',
 'particular',
 'Article',
 '62(1)(a)',
 'thereof,.',
 'Having',
 'called',
 'on',
 'Member',
 'States',
 'and',
 'other',
 'interested',
 'parties',
 'to',
 'submit',
 'their',
 'comments',
 'pursuant',
 'to',
 'those',
 'provisions',
 '(1),.',
 'Whereas:.',
 '1.',
 'PROCEDURE.',
 '1.1.',
 'PROCEDURE',
 'RELATED',
 'TO',
 'PIRAEUS',
 'BANK.',
 '(1).',


In [26]:
train_data[0].split(  )[:512]

['14.12.2017.',
 'EN.',
 'Official',
 'Journal',
 'of',
 'the',
 'European',
 'Union.',
 'L',
 '331/94.',
 'COMMISSION',
 'IMPLEMENTING',
 'DECISION',
 '(EU)',
 '2017/2320.',
 'of',
 '13',
 'December',
 '2017.',
 'on',
 'the',
 'equivalence',
 'of',
 'the',
 'legal',
 'and',
 'supervisory',
 'framework',
 'of',
 'the',
 'United',
 'States',
 'of',
 'America',
 'for',
 'national',
 'securities',
 'exchanges',
 'and',
 'alternative',
 'trading',
 'systems',
 'in',
 'accordance',
 'with',
 'Directive',
 '2014/65/EU',
 'of',
 'the',
 'European',
 'Parliament',
 'and',
 'of',
 'the',
 'Council.',
 '(Text',
 'with',
 'EEA',
 'relevance).',
 'THE',
 'EUROPEAN',
 'COMMISSION,.',
 'Having',
 'regard',
 'to',
 'the',
 'Treaty',
 'on',
 'the',
 'Functioning',
 'of',
 'the',
 'European',
 'Union,.',
 'Having',
 'regard',
 'to',
 'Directive',
 '2014/65/EU',
 'of',
 'the',
 'European',
 'Parliament',
 'and',
 'of',
 'the',
 'Council',
 'of',
 '15',
 'May',
 '2014',
 'on',
 'markets',
 'in',
 'financ

In [17]:
test_data=open( os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06", "test_data.tsv"  )  ).read().split( "\n" )

In [47]:
print( base64.b64decode( test_data[40].split( "\t" )[0]  ).decode( 'utf-8' )    )

15.3.2018
EN
Official Journal of the European Union
L 72/13
COMMISSION REGULATION (EU) 2018/400
of 14 March 2018
amending Regulation (EC) No 1126/2008 adopting certain international accounting standards in accordance with Regulation (EC) No 1606/2002 of the European Parliament and of the Council as regards International Accounting Standard (IAS) 40
(Text with EEA relevance)
THE EUROPEAN COMMISSION,
Having regard to the Treaty on the Functioning of the European Union,
Having regard to Regulation (EC) No 1606/2002 of the European Parliament and of the Council of 19 July 2002 on the application of international accounting standards (1), and in particular Article 3(1) thereof,
Whereas:
(1)
By Commission Regulation (EC) No 1126/2008 (2) certain international standards and interpretations that were in existence at 15 October 2008 were adopted.
(2)
On 8 December 2016, the International Accounting Standards Board (IASB) published the amendments to International Accounting Standard (IAS) 40 Inv

In [48]:
test_data[40].split( "\t" )[1]

'accepted'

### END

In [1421]:
length=[]
for text in all_articles:
    length.append( len( text.split( )  ) )

In [1426]:
np.std( length )

2140.8308963783843

In [ ]:
for article in all_articles:

    encoded_doc=b64encode(  text.encode() )
    f.write( f"{encoded_doc.decode()  }\t{ label_name }\t{label}\n" )

In [1393]:
len(xhtmls)

2742

In [1385]:
from random import randint
from random import seed
# seed random number generator

i=randint(0, 2742 )
print( i )

2257


#declined: <br>

In [1386]:

print( len( set (accepted ) ) )

249


In [1394]:
xhtmls[2375]

[{'id': 'fb91dcf2-63e8-53be-87b0-f486691df91d',
  'celex': ['32017D1519'],
  'title': ['Commission Implementing Decision (EU) 2017/1519 of 1 September 2017 amending the Annex to Implementing Decision (EU) 2017/247 on protective measures in relation to outbreaks of the highly pathogenic avian influenza in certain Member States (notified under document C(2017) 6056) (Text with EEA relevance. )'],
  'status': ['In force'],
  'type': ['Implementing decision'],
  'date': ['2017-09-01T00:00:00Z'],
  'url': ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32017D1519'],
  'eli': ['http://data.europa.eu/eli/dec_impl/2017/1519/oj'],
  'website': ['<span class="highlight">EURLEX</span>'],
  'various': ['C/2017/6056, OJ L 228, 2.9.2017, p. 1–18 (BG, ES, CS, DA, DE, ET, EL, EN, FR, HR, IT, LV, LT, HU, MT, NL, PL, PT, RO, SK, SL, FI, SV)'],
  'accepted_probability': [0.0],
  'pull': [False],
  '_version_': 1664110440769126400},
 {'content': '...',
  'content_type': 'xhtml'}]

In [1387]:
#uncertain <br>
[258, 116 , 8, 2660, 1697, 708, 2650, 1880, 770, 1825, 2583, 1254 ]

[258, 116, 8, 2660, 1697, 708, 2650, 1880, 770, 1825, 2583, 1254]

In [1388]:
decoded_content = base64.b64decode( xhtmls[i][1][ 'content' ] ).decode( 'utf-8' )
articles=clean_html( decoded_content )
print( "\n".join( articles ) )

4.3.2010
EN
Official Journal of the European Union
L 53/19
DECISION OF THE STANDING COMMITTEE OF THE EFTA STATES
No 1/2010/SC
of 28 January 2010
establishing an EEA Financial Mechanism Interim Committee 2009-2014
THE STANDING COMMITTEE OF THE EFTA STATES,
Having regard to the Agreement on the European Economic Area, as adjusted by the Protocol Adjusting the Agreement on the European Economic Area, hereinafter referred to as the EEA Agreement,
Having regard to the agreement to be concluded establishing a new EEA Financial Mechanism for the period 2009-2014,
Considering the agreement between the Kingdom of Norway and the European Community to establish a Norwegian Financial Mechanism for the period 2009-2014,
HAS DECIDED AS FOLLOWS:


Article 1
1.   An EEA Financial Mechanism Interim Committee 2009-2014, hereinafter referred to as the Interim Committee, which should be operative as soon as possible, is hereby established.
2.   The Interim Committee shall assist the EFTA States in prepari

In [1378]:
xhtmls[i]

[{'celex': ['32014R1105'],
  'title': ['Council Implementing Regulation (EU) No 1105/2014 of 20 October 2014 implementing Regulation (EU) No 36/2012 concerning restrictive measures in view of the situation in Syria'],
  'status': ['In force: This act has been changed. Current consolidated version: 21/10/2014'],
  'ELI': ['http://data.europa.eu/eli/reg_impl/2014/1105/oj'],
  'various': ['OJ L 301, 21.10.2014, p. 7–11 (BG, ES, CS, DA, DE, ET, EL, EN, FR, HR, IT, LV, LT, HU, MT, NL, PL, PT, RO, SK, SL, FI, SV), ELI: http://data.europa.eu/eli/reg_impl/2014/1105/oj'],
  'pdf_docs': ['https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32014R1105&from=EN'],
  'date': ['2014-10-20T00:00:00Z'],
  'dates': ['2014-10-20T00:00:00Z',
   '2014-10-21T00:00:00Z',
   '9999-01-01T00:00:00Z'],
  'dates_type': ['date of document',
   'date of effect',
   'date of end of validity'],
  'dates_info': ['n/a', 'Entry into force  Date pub. See Art 2', 'n/a'],
  'classifications_label': ['repression',

In [904]:
i

599

In [75]:
htmls[137]

[{'celex': ['31999L0067'],
  'title': ['Commission Directive 1999/67/EC of 28 June 1999 amending Directive 93/49/EEC setting out the schedule indicating the conditions to be met by ornamental plant propagating material and ornamental plants pursuant to Council Directive 91/682/EEC'],
  'status': ['In force'],
  'ELI': ['http://data.europa.eu/eli/dir/1999/67/oj'],
  'various': ['OJ L 164, 30.6.1999, p. 78–78 (ES, DA, DE, EL, EN, FR, IT, NL, PT, FI, SV)Special edition in Czech: Chapter 03 Volume 025 P. 452  - 452Special edition in Estonian: Chapter 03 Volume 025 P. 452  - 452Special edition in Latvian: Chapter 03 Volume 025 P. 452  - 452Special edition in Lithuanian: Chapter 03 Volume 025 P. 452  - 452Special edition in Hungarian Chapter 03 Volume 025 P. 452  - 452Special edition in Maltese: Chapter 03 Volume 025 P. 452  - 452Special edition in Polish: Chapter 03 Volume 025 P. 452  - 452Special edition in Slovak: Chapter 03 Volume 025 P. 452  - 452Special edition in Slovene: Chapter 03 V

In [44]:
decoded_content=base64.b64decode( json_file['content']  ).decode( 'utf-8' )

pdfs[0]

[{'id': '305e3053-76f8-5674-a7af-0a495b89c6b1',
  'celex': ['31986R0709'],
  'title': ['Commission Regulation (EEC) No 709/86 of 6 March 1986 fixing the minimum levies on the importation of olive oil and levies on the importation of other olive oil sector products'],
  'type': ['Regulation'],
  'date': ['1986-03-07T00:00:00Z'],
  'url': ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:31986R0709'],
  'eli': ['http://data.europa.eu/eli/reg/1986/709/oj'],
  'website': ['<span class="highlight">EURLEX</span>'],
  'various': ['OJ L 65, 7.3.1986, p. 6–8 (ES, DA, DE, EL, EN, FR, IT, NL, PT), ELI: http://data.europa.eu/eli/reg/1986/709/oj'],
  'accepted_probability': [0.0],
  'pull': [False],
  'created_at': ['2020-04-21T12:19:51.020Z'],
  'updated_at': ['2020-04-21T12:19:51.020Z'],
  '_version_': 1664594842071597056},
 {'content': 'CgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKTm8gTCA2NS82IE9mZmljaWFsIEpvdXJuYWwgb2YgdGhlIEV1cm9wZWFuIENvbW11bml0aWVzIDcuIDMgLiA4NgoKQ09NTUl

In [55]:
print( base64.b64decode( pdfs[0][1]['content'] ).decode( 'utf-8' )  )











































No L 65/6 Official Journal of the European Communities 7. 3 . 86

COMMISSION REGULATION (EEC) No 709/86
of 6 March 1986

fixing the minimum levies on the importation of olive oil and levies on the
importation of other olive oil sector products

the products concerned on the basis of the situation on
the world market and the Community market and of the
levy rates indicated by tenderers ;

Whereas in the collection of the levy, account should be
taken of the provisions in the Agreements between the
Community and certain third countries ; whereas in parti­
cular the levy applicable for those countries must be fixed
taking as a basis for calculation the levy to be collected on
imports from the other third countries ;

Whereas, with regard to Turkey and the Maghreb coun­
tries , the provisions of this Regulation should be without
prejudice to the additional amount to be determined in
accordance with the agreements between the Community
and these thi

In [61]:
print( base64.b64decode( xhtmls[0][1]['content'] ).decode( 'utf-8' )  )

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<!-- CONVEX # converter_version:3.2 # generated_on:20140124-0800 -->
<head>
   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>
   <link type="text/css" rel="stylesheet"
         href="http://publications.europa.eu/resource/presentation-style-sheet/oj/oj.css"/>
   <title>L_2008076EN.01003301.xml</title>
</head>
<body>
   <table width="100%" border="0" cellspacing="0" cellpadding="0">
      <col width="10%"/>
      <col width="10%"/>
      <col width="60%"/>
      <col width="20%"/>
      <tbody>
         <tr>
            <td>
               <p class="hd-date">19.3.2008   </p>
            </td>
            <td>
               <p class="hd-lg">EN</p>
            </td>
            <td>
               <p class="hd-ti">Official Journal of the European Union</p>
            </td>
            <td>
               <p class="hd-o

In [39]:
from collections import Counter

content_types=[]
for jsonl in all_jsons:
    content_types.append( jsonl[1]['content_type']  )
    
Counter( content_types )

Counter({'xhtml': 2742, 'html': 5623, 'pdf': 6787})

In [38]:
import collections
collections.collections

array(['html', 'pdf', 'xhtml'], dtype='<U5')

In [30]:
i=0
for json in all_jsons:
    if len(json)>1:
        i=i+1

In [31]:
i

15152

In [29]:
len(all_jsons)

15152

In [8]:
print(all_articles[110])

9.11.2018
EN
Official Journal of the European Union
L 281/1
COMMISSION IMPLEMENTING REGULATION (EU) 2018/1627
of 9 October 2018
amending Implementing Regulation (EU) No 680/2014 as regards prudent valuation for supervisory reporting
(Text with EEA relevance)
THE EUROPEAN COMMISSION,
Having regard to the Treaty on the Functioning of the European Union,
Having regard to Regulation (EU) No 575/2013 of the European Parliament and of the Council of 26 June 2013 on prudential requirements for credit institutions and investment firms and amending Regulation (EU) No 648/2012 (1) and in particular the fourth subparagraph of Article 99(5), the fourth subparagraph of Article 99(6), the third subparagraph of Article 394(4) the fourth subparagraph of Article 415(3) and the third subparagraph of Article 430(2) thereof,
Whereas:
(1)
Commission Implementing Regulation (EU) No 680/2014 (2) specifies the modalities according to which institutions are required to report information relevant to their comp

In [4]:
length=[]
for article in all_articles:
    length.append(len( article.split(  ) ) )

In [5]:
np.mean( length )

3215.4139344262294

In [6]:
np.std( length )

6281.226895787415

In [7]:
np.max(length)

66114

In [8]:
np.min(length)

267

test


In [24]:
print( all_articles[110] )

9.11.2018
EN
Official Journal of the European Union
L 281/1
COMMISSION IMPLEMENTING REGULATION (EU) 2018/1627
of 9 October 2018
amending Implementing Regulation (EU) No 680/2014 as regards prudent valuation for supervisory reporting
(Text with EEA relevance)
THE EUROPEAN COMMISSION,
Having regard to the Treaty on the Functioning of the European Union,
Having regard to Regulation (EU) No 575/2013 of the European Parliament and of the Council of 26 June 2013 on prudential requirements for credit institutions and investment firms and amending Regulation (EU) No 648/2012 (1) and in particular the fourth subparagraph of Article 99(5), the fourth subparagraph of Article 99(6), the third subparagraph of Article 394(4) the fourth subparagraph of Article 415(3) and the third subparagraph of Article 430(2) thereof,
Whereas:
(1)
Commission Implementing Regulation (EU) No 680/2014 (2) specifies the modalities according to which institutions are required to report information relevant to their comp

In [11]:
for i, leng in enumerate( length ):
    print( i, leng )

0 2249
1 1679
2 2364
3 2446
4 2213
5 1565
6 2133
7 1479
8 1478
9 1429
10 2045
11 3015
12 639
13 4089
14 1035
15 4401
16 1997
17 1982
18 2325
19 1345
20 1479
21 3578
22 1014
23 2353
24 753
25 4287
26 4282
27 3106
28 3832
29 2775
30 350
31 1261
32 298
33 287
34 592
35 799
36 453
37 655
38 66114
39 22782
40 2547
41 6895
42 1043
43 1047
44 1638
45 927
46 1291
47 4291
48 1134
49 437
50 657
51 427
52 1282
53 595
54 297
55 1030
56 655
57 574
58 5161
59 1941
60 2906
61 655
62 1491
63 5582
64 4184
65 2598
66 5290
67 890
68 1681
69 433
70 654
71 267
72 291
73 657
74 414
75 274
76 675
77 602
78 568
79 901
80 608
81 968
82 2063
83 1374
84 17040
85 4284
86 1958
87 1648
88 1387
89 816
90 843
91 4975
92 695
93 1014
94 1416
95 507
96 4320
97 655
98 2524
99 2954
100 1390
101 2353
102 1781
103 2022
104 2224
105 8529
106 455
107 890
108 445
109 986
110 11101
111 2560
112 1641
113 1668
114 1923
115 2287
116 2445
117 11120
118 4001
119 405
120 398
121 5534
122 1395
123 411
124 2230
125 1705
126 752
127 117

In [2]:
import glob
import json
DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_30_04"
files=glob.glob( os.path.join( DATA_DIR, "EURLEX" , "*jsonl" )  )

all_jsons=[]

for file in files:
    jsonl=open( file ).read().rstrip("\n").split( "\n" )
    json_file = [json.loads( json_line ) for json_line in jsonl]
    all_jsons.append( json_file )


In [49]:
eurovocs=[]
for json in all_jsons:
    metadata=json[0]
    if 'classifications_type' in metadata.keys() and "classifications_label" in metadata.keys():
        if "eurovoc descriptor" in metadata[ 'classifications_type' ]:
            eurovoc=[]
            for i, voc in enumerate( metadata['classifications_label'] ):
                if metadata['classifications_type'][i]=="eurovoc descriptor":
                    eurovoc.append( voc )
            eurovocs.append( ( eurovoc , json ) )

In [53]:
eurovoc_dgfisma=open( "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/eurovoc_rules/eurovoc_in_DGFISMA"  ).read().strip( "\n" ).split("\n")
eurovoc_dgfisma=set(eurovoc_dgfisma)

In [64]:
i=0
for eurovoc, json in eurovocs:
    for word in eurovoc:
        if word.lower() in eurovoc_dgfisma:
            i=i+1
            print(word)
            break
            

financial control
financial control
credit guarantee
insurance
financial control
credit institution
accounting
financial control
credit institution
credit institution
accounting
financial solvency
stock exchange
insurance
financial instrument
credit institution
financial aid
accounting
financial aid
pension scheme
shareholding
credit institution
shareholding
securities
financial intervention
accounting
credit institution
securities
credit institution
shareholding
insurance contract
credit guarantee
pension scheme
securities
financial instrument
financial institution
investment company
shareholding
investment company
financial institution
financial aid
shareholding
accounting
financial market
investment company
financial control
credit institution
financial instrument
shareholding
financial control
consumer credit
credit institution
financial aid
credit guarantee
financial intervention
shareholding
capital market
financial institution
pension scheme
financial aid
insurance
investment co

In [6]:
directory_codes=[]
for json in all_jsons:
    metadata=json[0]
    if 'classifications_type' in metadata.keys() and "classifications_label" in metadata.keys():
        if "directory code" in metadata[ 'classifications_type' ]:
            directory_code=[]
            for i, code in enumerate( metadata['classifications_code'] ):
                if metadata['classifications_type'][i]=="directory code":
                    directory_code.append( code )
            directory_codes.append( ( directory_code , json ) )

In [7]:
len(directory_codes)

5767

In [25]:
i=0
for directory_code, _ in directory_codes:
    for code in directory_code:
        if code[:8]=='06202025':
            print(code)
            i=i+1
            break

06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025
06202025


In [26]:
i

28

In [9]:
a='test'
a[:2]

'te'

In [63]:
'Fina I'.lower()

'fina i'

In [25]:
all_jsons[9][0]['classifications_label']

['atmospheric pollution',
 'environmental protection',
 'motor vehicle pollution',
 'clean technology',
 'road transport',
 'motor vehicle',
 'climate change',
 'greenhouse gas',
 'reduction of gas emissions',
 'energy efficiency',
 'Environment',
 'Transport',
 'Transport policy',
 'General',
 'Industrial policy and internal market',
 'Internal market: approximation of laws',
 'Motor vehicles',
 'Environment, consumers and health protection',
 'Environment']

In [29]:
all_jsons[9][0]['classifications_type']

['eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'eurovoc descriptor',
 'subject matter',
 'subject matter',
 'directory code',
 'directory code',
 'directory code',
 'directory code',
 'directory code',
 'directory code',
 'directory code']

In [40]:
eurovocs

['atmospheric pollution',
 'environmental protection',
 'motor vehicle pollution',
 'clean technology',
 'road transport',
 'motor vehicle',
 'climate change',
 'greenhouse gas',
 'reduction of gas emissions',
 'energy efficiency']